In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width: 98% !important; }</style>"))
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [ ]:
import sys
import os
import re
import numpy as np
import pandas as pd

from tqdm.notebook import tqdm
from sklearn.metrics import f1_score, confusion_matrix

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
sns.set()

In [ ]:
from lib import util, fitter, visualize

In [ ]:
datapath = 'SemEval_2022_Task2-idiomaticity/SubTaskA/Data'
testpath = 'SemEval_2022_Task2-idiomaticity/SubTaskA/TestData'

In [ ]:
frames = util.load_csv_dataframes(datapath)
tframes = util.load_csv_dataframes(testpath)

In [ ]:
zdf = frames['train_zero_shot.csv']
odf = frames['train_one_shot.csv']
ddf = frames['dev.csv']
ddf_gold = frames['dev_gold.csv']
edf = frames['eval.csv']
tdf = tframes['test.csv']

In [ ]:
zdf_bt3 = pd.read_pickle('data/zdf_bt3_20220104_1.pkl')
ddf_bt3 = pd.read_pickle('data/ddf_bt3_20220104_1.pkl')
edf_bt3 = pd.read_pickle('data/edf_bt3_20220105_1.pkl')
tdf_bt3 = pd.read_pickle('data/tdf_bt3_20220111_1.pkl')

In [ ]:
zdf_t = fitter.get_trainable(zdf_bt3)
ddf_t = fitter.get_trainable(ddf_bt3)
zdf_t['Label'] = zdf_bt3['Label']
ddf_t['Label'] = ddf_gold['Label']
zdf_t['Set'] = 'Train'
ddf_t['Set'] = 'Dev'

In [ ]:
pairplot = sns.pairplot(zdf_t.drop(['Quotes', 'Hassub', 'Caps', 'Trans'], axis=1), hue='Label')

In [ ]:
zdf_bt3.groupby(['Language','Label'])['DataID'].count()

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(20,5))
sns.boxplot(ax=axes[0], data=zdf_t, y='Sentiment', hue='Label', x=zdf_bt3['Language'])
sns.boxplot(ax=axes[1], data=zdf_t, y='Nextdiff', hue='Label', x=zdf_bt3['Language'])
sns.boxplot(ax=axes[2], data=zdf_t, y='Prevdiff', hue='Label', x=zdf_bt3['Language'])
sns.boxplot(ax=axes[3], data=zdf_t, y='MWEdiff', hue='Label', x=zdf_bt3['Language'])

In [ ]:
fig2, axes2 = plt.subplots(1, 3, figsize=(15,5))
sns.boxplot(ax=axes2[0], data=zdf_t, y='Top score', hue='Label', x=zdf_bt3['Language'])
sns.boxplot(ax=axes2[1], data=zdf_t, y='Top score 1', hue='Label', x=zdf_bt3['Language'])
sns.boxplot(ax=axes2[2], data=zdf_t, y='Top score 2', hue='Label', x=zdf_bt3['Language'])
# sns.boxplot(ax=axes2[3], data=zdf_t, y='FoundScore', hue='Label', x=zdf_bt3['Language'])

In [ ]:
comb_t = pd.concat([zdf_t, ddf_t], ignore_index=True)
comb_bt3 = pd.concat([zdf_bt3, ddf_bt3], ignore_index=True)

In [ ]:
fig3, axes3 = plt.subplots(1, 2, figsize=(10,5))
sns.boxplot(ax=axes3[0], data=comb_t, y='Sentiment', x='Label', hue='Set')
sns.boxplot(ax=axes3[1], data=comb_t, y='Sentiment', hue='Set', x=comb_bt3['Language'])
# sns.boxplot(ax=axes3[2], data=comb_t, y='Sentiment', x=comb_bt3['Language'], hue='Label')

In [ ]:
fig4, axes4 = plt.subplots(3, 4, sharex=True, figsize=(20,15))
row=0
col=0
for column in zdf_t.drop(['Label', 'Hassub', 'Caps', 'Quotes', 'Trans', 'Set'], axis=1):
    # print(row,col)
    sns.boxplot(ax=axes4[row, col], y=zdf_t[column], hue=zdf_t['Label'], x=zdf_bt3['Language'])
    col += 1
    if col >= axes4.shape[1]:
        row += 1
        col = 0

In [ ]:
fig5, axes5 = plt.subplots(3, 4, figsize=(20,15))
row=0
col=0
for column in comb_t.drop(['Label', 'Hassub', 'Caps', 'Quotes', 'Trans', 'Set'], axis=1):
    # print(row,col)
    sns.boxplot(ax=axes5[row, col], y=comb_t[column], hue=comb_t['Label'], x=comb_t['Set'])
    col += 1
    if col >= axes5.shape[1]:
        row += 1
        col = 0

In [ ]:
sns.jointplot(data=zdf_bt3, x='Sentiment', y='Top score', hue='Label')

In [ ]:
fig6, axes6 = plt.subplots(1, 4, figsize=(20,4))
sns.countplot(ax=axes6[0], data=zdf_bt3, x='Hassub', hue='Label')
sns.countplot(ax=axes6[1], data=zdf_bt3, x='Quotes', hue='Label')
sns.countplot(ax=axes6[2], data=zdf_bt3, x='Caps', hue='Label')
sns.countplot(ax=axes6[3], data=zdf_bt3, x='Trans', hue='Label')
fig7, axes7 = plt.subplots(1, 4, figsize=(20,4))
sns.countplot(ax=axes7[0], data=zdf_bt3[zdf_bt3['Language'] == 'EN'], x='Hassub', hue='Label')
sns.countplot(ax=axes7[1], data=zdf_bt3[zdf_bt3['Language'] == 'EN'], x='Quotes', hue='Label')
sns.countplot(ax=axes7[2], data=zdf_bt3[zdf_bt3['Language'] == 'EN'], x='Caps', hue='Label')
sns.countplot(ax=axes7[3], data=zdf_bt3[zdf_bt3['Language'] == 'EN'], x='Trans', hue='Label')
fig8, axes8 = plt.subplots(1, 4, figsize=(20,4))
sns.countplot(ax=axes8[0], data=zdf_bt3[zdf_bt3['Language'] == 'PT'], x='Hassub', hue='Label', hue_order=['0','1'])
sns.countplot(ax=axes8[1], data=zdf_bt3[zdf_bt3['Language'] == 'PT'], x='Quotes', hue='Label', hue_order=['0','1'])
sns.countplot(ax=axes8[2], data=zdf_bt3[zdf_bt3['Language'] == 'PT'], x='Caps', hue='Label', hue_order=['0','1'])
sns.countplot(ax=axes8[3], data=zdf_bt3[zdf_bt3['Language'] == 'PT'], x='Trans', hue='Label', hue_order=['0','1'])
# FIXME: add horizontal labels (Top: Both, Middle: English, Bottom: Portuguese)

In [ ]:
fig9, axes9 = plt.subplots(3, 4, figsize=(20,15))
row=0
col=0
for column in zdf_t.drop(['Label', 'Hassub', 'Caps', 'Quotes', 'Trans', 'Set'], axis=1):
    # print(row,col)
    sns.violinplot(ax=axes9[row, col], y=zdf_t[column], hue=zdf_t['Label'], x=zdf_bt3['Language'], split=True)
    col += 1
    if col >= axes9.shape[1]:
        row += 1
        col = 0

In [ ]:
fig10, axes10 = plt.subplots(1, 4, figsize=(20,4))
sns.countplot(ax=axes10[0], data=zdf_bt3, hue='Hassub', x='Label')
sns.countplot(ax=axes10[1], data=zdf_bt3, hue='Quotes', x='Label')
sns.countplot(ax=axes10[2], data=zdf_bt3, hue='Caps', x='Label')
sns.countplot(ax=axes10[3], data=zdf_bt3, hue='Trans', x='Label')
fig11, axes11 = plt.subplots(1, 4, figsize=(20,4))
sns.countplot(ax=axes11[0], data=zdf_bt3[zdf_bt3['Language'] == 'EN'], hue='Hassub', x='Label')
sns.countplot(ax=axes11[1], data=zdf_bt3[zdf_bt3['Language'] == 'EN'], hue='Quotes', x='Label')
sns.countplot(ax=axes11[2], data=zdf_bt3[zdf_bt3['Language'] == 'EN'], hue='Caps', x='Label')
sns.countplot(ax=axes11[3], data=zdf_bt3[zdf_bt3['Language'] == 'EN'], hue='Trans', x='Label')
fig12, axes12 = plt.subplots(1, 4, figsize=(20,4))
sns.countplot(ax=axes12[0], data=zdf_bt3[zdf_bt3['Language'] == 'PT'], hue='Hassub', x='Label', order=['0','1'])
sns.countplot(ax=axes12[1], data=zdf_bt3[zdf_bt3['Language'] == 'PT'], hue='Quotes', x='Label', order=['0','1'])
sns.countplot(ax=axes12[2], data=zdf_bt3[zdf_bt3['Language'] == 'PT'], hue='Caps', x='Label', order=['0', '1'])
sns.countplot(ax=axes12[3], data=zdf_bt3[zdf_bt3['Language'] == 'PT'], hue='Trans', x='Label', order=['0', '1'])

In [ ]:
fig11, axes11 = plt.subplots(1, 2, figsize=(8,4))
sns.kdeplot(ax=axes11[0],data=zdf_bt3[zdf_bt3['Language'] == 'EN'], hue='Label', x='Sentiment', fill=True)
sns.kdeplot(ax=axes11[1],data=zdf_bt3[zdf_bt3['Language'] == 'PT'], hue='Label', x='Sentiment', fill=True)

In [ ]:
heatmap_z = sns.clustermap(zdf_t.corr(), cbar_pos=(.1, .5, .03, .2), cmap="Blues")
heatmap_z.ax_row_dendrogram.remove()

In [ ]:
cf = pd.crosstab(zdf_bt3['Trans'], zdf_bt3['Label'])
sns.heatmap(cf, annot=True, cmap='Blues', fmt='d')

In [ ]:
ddf_res = util.load_df('data/ddf_sub_20220121_1.csv')
ddf_res2 = util.load_df('data/ddf_sub_20220121_2.csv')

In [ ]:
ddf_pred_labels = ddf_res[ddf_res['Setting'] == 'zero_shot']['Label']
ddf_cf = confusion_matrix(ddf_pred_labels, ddf_gold['Label'])
cfax1 = sns.heatmap(ddf_cf, annot=True, cmap='Blues', fmt='d')
cfax1.set(ylabel='Actual label', xlabel='Predicted label')

Confusion matrix for the sbert+feature model.

In [ ]:
ddf_pred_labels2 = ddf_res2[ddf_res2['Setting'] == 'zero_shot']['Label']
ddf_cf2 = confusion_matrix(ddf_pred_labels2, ddf_gold['Label'])
counts2 = [ v for v in ddf_cf2.flatten() ]
pct2 = [ v for v in ddf_cf2.flatten()/np.sum(ddf_cf2) ]
labels2 = [ "%s\n%.1f%%" % (v1,100*v2) for v1, v2 in zip(counts2, pct2) ]
labels2 = np.asarray(labels2).reshape(2,2)
cfax2 = sns.heatmap(ddf_cf2, annot=labels2, cmap='Blues', fmt='')
cfax2.set(ylabel='Actual label', xlabel='Predicted label')

Confusion matrix for the bert+feature model.

In [ ]:
ddf_pred_labels3 = ddf_res2[(ddf_res2['Setting'] == 'zero_shot') & (ddf_bt3['Language'] == 'EN')]['Label']
ddf_cf3 = confusion_matrix(ddf_pred_labels3, ddf_gold[ddf_gold['Language'] == 'EN']['Label'])
counts3 = [ v for v in ddf_cf3.flatten() ]
pct3 = [ v for v in ddf_cf3.flatten()/np.sum(ddf_cf3) ]
labels3 = [ "%s\n%.1f%%" % (v1,100*v2) for v1, v2 in zip(counts3, pct3) ]
labels3 = np.asarray(labels3).reshape(2,2)
cfax3 = sns.heatmap(ddf_cf3, annot=labels3, cmap='Blues', fmt='')
cfax3.set(ylabel='Actual label', xlabel='Predicted label')

Confusion matrix for the bert+feature model, English.

In [ ]:
ddf_pred_labels4 = ddf_res2[(ddf_res2['Setting'] == 'zero_shot') & (ddf_bt3['Language'] == 'PT')]['Label']
ddf_cf4 = confusion_matrix(ddf_pred_labels4, ddf_gold[ddf_gold['Language'] == 'PT']['Label'])
counts4 = [ v for v in ddf_cf4.flatten() ]
pct4 = [ v for v in ddf_cf4.flatten()/np.sum(ddf_cf4) ]
labels4 = [ "%s\n%.1f%%" % (v1,100*v2) for v1, v2 in zip(counts4, pct4) ]
labels4 = np.asarray(labels4).reshape(2,2)
cfax4 = sns.heatmap(ddf_cf4, annot=labels4, cmap='Blues', fmt='')
cfax4.set(ylabel='Actual label', xlabel='Predicted label')